In [1]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import os
import sys

# Añadimos la carpeta principal al path para encontrar nuestros módulos
sys.path.insert(0, os.path.abspath('..'))

# Importamos TODAS nuestras funciones de ingeniería de características
from src.feature_engineering import add_pythagorean_expectation, EloRatingSystem, add_elo_ratings, add_rolling_win_percentage

print("--- Fase 2: Mejora con Características Dinámicas (Elo y Forma) ---")

# 1. Cargar el dataset
# Necesitamos añadir los nombres de los equipos y la fecha para estas nuevas características
# Vamos a tener que volver a generar nuestro dataset con más información.
# Por ahora, vamos a simularlo para no tener que esperar.
# (En el siguiente paso, te daré el script 'build_dataset_v2.py' actualizado)

# --- SIMULACIÓN TEMPORAL ---
# En un paso futuro, reemplazaremos esto con la carga de un dataset más rico
csv_path = os.path.join('..', 'data', 'historical_game_data_with_features.csv')
df = pd.read_csv(csv_path)
# Añadimos columnas falsas de nombre y fecha para que el código funcione
df['game_date'] = pd.to_datetime('2023-01-01') + pd.to_timedelta(df.index, unit='D')
df['h_team_name'] = 'Team_A'
df['v_team_name'] = 'Team_B'
df.loc[df.index % 2 == 0, 'h_team_name'] = 'Team_C'
df.loc[df.index % 2 == 0, 'v_team_name'] = 'Team_D'
# --- FIN DE SIMULACIÓN ---

print(f"Dataset original cargado con {df.shape} filas.")

# 2. Aplicar TODA la ingeniería de características en secuencia
df_v3 = add_pythagorean_expectation(df)
df_v3 = add_rolling_win_percentage(df_v3) # La forma se calcula antes de Elo

# 3. "Entrenar" el sistema Elo
# Elo debe procesar los partidos en orden cronológico para ser preciso.
elo_sys = EloRatingSystem()
# Creamos columnas para el ganador y perdedor para facilitar el entrenamiento de Elo
df_v3['winner'] = df_v3.apply(lambda row: row['h_team_name'] if row['target'] == 1 else row['v_team_name'], axis=1)
df_v3['loser'] = df_v3.apply(lambda row: row['v_team_name'] if row['target'] == 1 else row['h_team_name'], axis=1)

for index, row in df_v3.sort_values('game_date').iterrows():
    elo_sys.update_ratings(row['winner'], row['loser'])

# Ahora que el sistema Elo está "entrenado", aplicamos las calificaciones al dataset
df_v3 = add_elo_ratings(df_v3, elo_sys)

# 4. Preparar los datos para el modelo v3
y = df_v3['target']
features_v3 = ['h_team_wins_season', 'h_team_losses_season', 'v_team_wins_season', 'v_team_losses_season', 
               'pythag_diff', 'elo_diff', 'win_pct_roll_diff']
X = df_v3[features_v3]

print("\nCaracterísticas finales para el modelo v3:")
print(X.columns.tolist())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. Entrenar y Evaluar el modelo v3
print("\nEntrenando modelo v3 con características dinámicas...")
model_v3 = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
model_v3.fit(X_train, y_train)
print("¡Modelo v3 entrenado!")

predictions_v3 = model_v3.predict(X_test)
accuracy_v3 = accuracy_score(y_test, predictions_v3)

print("\n" + "="*60)
print(f"  PRECISIÓN LÍNEA BASE (v1): 0.5585")
print(f"  PRECISIÓN CON PITÁGORAS (v2): 0.5766")
print(f"  PRECISIÓN CON ELO Y FORMA (v3): {accuracy_v3:.4f}")
print("="*60)

improvement_v3 = accuracy_v3 - 0.5766
print(f"\n¡Mejora sobre la versión anterior: {improvement_v3:+.2%}!")
total_improvement = accuracy_v3 - 0.5585
print(f"¡Mejora total sobre la línea base: {total_improvement:+.2%}!")

--- Fase 2: Mejora con Características Dinámicas (Elo y Forma) ---
Dataset original cargado con (2476, 8) filas.
--- Aplicando Ingeniería de Características: Expectativa Pitagórica ---
¡Característica 'pythag_diff' creada con éxito!
--- Aplicando Ingeniería de Características: Forma Reciente (Medias Móviles) ---
¡Característica 'win_pct_roll_diff' creada con éxito!
--- Aplicando Ingeniería de Características: Calificaciones Elo ---
¡Característica 'elo_diff' creada con éxito!

Características finales para el modelo v3:
['h_team_wins_season', 'h_team_losses_season', 'v_team_wins_season', 'v_team_losses_season', 'pythag_diff', 'elo_diff', 'win_pct_roll_diff']

Entrenando modelo v3 con características dinámicas...
¡Modelo v3 entrenado!

  PRECISIÓN LÍNEA BASE (v1): 0.5585
  PRECISIÓN CON PITÁGORAS (v2): 0.5766
  PRECISIÓN CON ELO Y FORMA (v3): 0.6190

¡Mejora sobre la versión anterior: +4.24%!
¡Mejora total sobre la línea base: +6.05%!
